In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [1]:
import os

# 设置HuggingFace镜像站
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HUGGINGFACE_CO_URL_HOME"] = "https://hf-mirror.com"
# 用来关闭 Xet 下载路径。关闭后将走常规下载（此时你设置的镜像站才真正生效）
os.environ["HF_HUB_DISABLE_XET"] = "1"   # 关键开关：禁用 Xet/CAS

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

In [2]:
# 加载数据
with open('mfd.md') as f:
    all_text = f.read()
# texts = all_text.split('\n\n')
texts = all_text.split('#### ')
len(texts)

27

In [3]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [4]:
# 适合处理英文
# from pymilvus import model as milvus_model
# embedding_model = milvus_model.DefaultEmbeddingFunction()

# 适合处理中文
from pymilvus.model.dense import SentenceTransformerEmbeddingFunction
# 效果不行
# embedding_model = SentenceTransformerEmbeddingFunction()
# BAAI/bge-small-zh-v1.5效果不错，BAAI/bge-large-zh-v1.5待测试
embedding_model = SentenceTransformerEmbeddingFunction(model_name="BAAI/bge-small-zh-v1.5")

result = embedding_model.encode_queries(["你好，世界"])
embedding_dim = len(result[0])
print(embedding_dim)
print(result[0][:10])

/root/miniconda3/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


512
[-0.00623569  0.05621756  0.0564207   0.01282544  0.0287137  -0.02857026
 -0.03341608  0.00869474 -0.01237282  0.05736669]


In [5]:
#删除homework.db
if os.path.exists("./homework.db"):
    os.remove("./homework.db")
    os.remove("./.homework.db.lock")

In [6]:
from pymilvus import MilvusClient
milvus_client = MilvusClient('./homework.db')
collection_name = 'homework'

In [7]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

In [8]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(texts)

for i, line in enumerate(tqdm(texts, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 27/27 [00:00<00:00, 212868.81it/s]


{'insert_count': 27, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26], 'cost': 0}

In [12]:
# question = "合同的权利义务终止"
question = "哪些情形会造成合同的权利义务终止"
# question = "拾得遗失物，如何处理？"
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [13]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u7b2c\u56db\u7ae0 \u5408\u540c\u7684\u6743\u5229\u4e49\u52a1\u7ec8\u6b62\n\n**\u7b2c\u4e94\u767e\u4e94\u5341\u4e03\u6761** \u6709\u4e0b\u5217\u60c5\u5f62\u4e4b\u4e00\u7684\uff0c\u5408\u540c\u7684\u6743\u5229\u4e49\u52a1\u7ec8\u6b62\uff1a\n\uff08\u4e00\uff09\u503a\u52a1\u5df2\u7ecf\u5c65\u884c\uff1b\n\uff08\u4e8c\uff09\u5408\u540c\u89e3\u9664\uff1b\n\uff08\u4e09\uff09\u503a\u52a1\u76f8\u4e92\u62b5\u9500\uff1b\n\uff08\u56db\uff09\u503a\u52a1\u4eba\u4f9d\u6cd5\u5c06\u6807\u7684\u7269\u63d0\u5b58\uff1b\n\uff08\u4e94\uff09\u503a\u6743\u4eba\u514d\u9664\u503a\u52a1\uff1b\n\uff08\u516d\uff09\u503a\u6743\u503a\u52a1\u540c\u5f52\u4e00\u4eba\uff1b\n\uff08\u4e03\uff09\u6cd5\u5f8b\u89c4\u5b9a\u6216\u8005\u5f53\u4e8b\u4eba\u7ea6\u5b9a\u7ec8\u6b62\u7684\u5176\u4ed6\u60c5\u5f62\u3002\n\n**\u7b2c\u4e94\u767e\u4e94\u5341\u516b\u6761** \u5408\u540c\u7ec8\u6b62\u540e\uff0c\u5f53\u4e8b\u4eba\u5e94\u5f53\u9075\u5faa\u8bda\u4fe1\u539f\u5219\uff0c\u6839\u636e\u4ea4\u6613\u4e60\u60ef\u5c65\u

In [14]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)
context

'第四章 合同的权利义务终止\n\n**第五百五十七条** 有下列情形之一的，合同的权利义务终止：\n（一）债务已经履行；\n（二）合同解除；\n（三）债务相互抵销；\n（四）债务人依法将标的物提存；\n（五）债权人免除债务；\n（六）债权债务同归一人；\n（七）法律规定或者当事人约定终止的其他情形。\n\n**第五百五十八条** 合同终止后，当事人应当遵循诚信原则，根据交易习惯履行通知、协助、保密等义务。\n\n**第五百五十九条** 合同的权利义务终止后，当事人应当履行清理义务。\n\n**第五百六十条** 当事人一方依照法律规定或者合同约定解除合同的，合同的权利义务终止。\n\n**第五百六十一条** 当事人一方不履行合同义务或者履行合同义务不符合约定的，对方有权请求其承担违约责任。\n\n**第五百六十二条** 有下列情形之一的，当事人可以解除合同：\n（一）因不可抗力致使不能实现合同目的；\n（二）在履行期限届满前，一方明确表示或者以自己的行为表明不履行主要债务；\n（三）一方迟延履行主要债务，经催告后在合理期限内仍未履行；\n（四）一方迟延履行债务或者有其他违约行为致使不能实现合同目的；\n（五）法律规定的其他情形。\n\n**第五百六十三条** 当事人可以约定一方解除合同的事由。解除合同的事由发生时，解除权人可以解除合同。\n\n**第五百六十四条** 法律规定或者当事人约定解除权行使期限，期限届满当事人不行使的，该权利消灭。\n法律没有规定或者当事人没有约定解除权行使期限，经对方催告后在合理期限内不行使的，该权利消灭。\n\n**第五百六十五条** 当事人一方解除合同的，应当通知对方。合同自通知到达对方时解除；通知载明债务人在一定期限内不履行债务则合同自动解除，债务人在该期限内未履行债务的，合同自期限届满时解除。\n对方有异议的，可以请求人民法院或者仲裁机构确认解除行为的效力。\n\n**第五百六十六条** 合同解除后，尚未履行的，终止履行；已经履行的，根据履行情况和合同性质，当事人可以请求恢复原状或者采取其他补救措施，并有权请求赔偿损失。\n合同因违约解除的，解除权人可以请求违约方承担违约责任。\n\n**第五百六十七条** 合同的权利义务终止后，当事人应当履行清理义务。\n合同终止后，当事人可以约定合同终止后的权利义务。\n法律、行政法规对合同终

In [15]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
<context>
{context}
</context>
<question>
{question}
</question>
"""
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。\n<context>\n第四章 合同的权利义务终止\n\n**第五百五十七条** 有下列情形之一的，合同的权利义务终止：\n（一）债务已经履行；\n（二）合同解除；\n（三）债务相互抵销；\n（四）债务人依法将标的物提存；\n（五）债权人免除债务；\n（六）债权债务同归一人；\n（七）法律规定或者当事人约定终止的其他情形。\n\n**第五百五十八条** 合同终止后，当事人应当遵循诚信原则，根据交易习惯履行通知、协助、保密等义务。\n\n**第五百五十九条** 合同的权利义务终止后，当事人应当履行清理义务。\n\n**第五百六十条** 当事人一方依照法律规定或者合同约定解除合同的，合同的权利义务终止。\n\n**第五百六十一条** 当事人一方不履行合同义务或者履行合同义务不符合约定的，对方有权请求其承担违约责任。\n\n**第五百六十二条** 有下列情形之一的，当事人可以解除合同：\n（一）因不可抗力致使不能实现合同目的；\n（二）在履行期限届满前，一方明确表示或者以自己的行为表明不履行主要债务；\n（三）一方迟延履行主要债务，经催告后在合理期限内仍未履行；\n（四）一方迟延履行债务或者有其他违约行为致使不能实现合同目的；\n（五）法律规定的其他情形。\n\n**第五百六十三条** 当事人可以约定一方解除合同的事由。解除合同的事由发生时，解除权人可以解除合同。\n\n**第五百六十四条** 法律规定或者当事人约定解除权行使期限，期限届满当事人不行使的，该权利消灭。\n法律没有规定或者当事人没有约定解除权行使期限，经对方催告后在合理期限内不行使的，该权利消灭。\n\n**第五百六十五条** 当事人一方解除合同的，应当通知对方。合同自通知到达对方时解除；通知载明债务人在一定期限内不履行债务则合同自动解除，债务人在该期限内未履行债务的，合同自期限届满时解除。\n对方有异议的，可以请求人民法院或者仲裁机构确认解除行为的效力。\n\n**第五百六十六条** 合同解除后，尚未履行的，终止履行；已经履行的，根据履行情况和合同性质，当事人可以请求恢复原状或者采取其他补救措施，并有权请求赔偿损失。\n合同因违约解除的，解除权人可以请求违约方承担违约责任。\n\n**第五百六十

In [16]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据《context》中**第五百五十七条**的规定，合同的权利义务在以下情形下终止：  

1. **债务已经履行**  
2. **合同解除**（包括约定解除或法定解除，如不可抗力、违约等情形）  
3. **债务相互抵销**（需标的物种类、品质相同且无禁止抵销约定）  
4. **债务人依法将标的物提存**  
5. **债权人免除债务**（全部或部分免除）  
6. **债权债务同归一人**（即混同，但不得损害第三人利益）  
7. **法律规定或当事人约定终止的其他情形**  

此外，**第五百六十条**补充说明，若一方依法或依约定解除合同，权利义务也终止。其他条款（如第五百六十二条）进一步细了解除合同的具体情形（如违约、不可抗力等）。
